## Cats and Dogs 이진 분류

### 이진분류 프로젝트

#### 개요

-  목표 : CNN으로 개와 고양이 이미지 분류 이진 분류기

#### Kaggle 데이터 다운로드 
- Kaggle이란? 
    - https://www.kaggle.com/ 
    - 예측모델 및 분석 대회 플랫폼. 2017년 구글이 인수 

1. Kaggle 계정 만들기
2. 설정진입
    - https://www.kaggle.com/settings
    - API > Create New Token 클릭 Kaggle.json 다운로드 

3. 파이썬 Kaggle 패기지 설치
!pip install kaggle

4. kaggle.json 파일 위치
    - 리눅스/유닉스 : ~/.kaggle/kaggle.json
    - 윈도우 : %USERPROFILE%\.kaggle
        - 사용자 폴더 아래
        - .kaggle 파일을 만든다 
        - 다운받은 Kaggle.json 토큰을 .kaggle 파일에 집어넣는다

5. https://www.kaggle.com/competitions/dogs-vs-cats/overview
    - Data

In [1]:
!pip install kaggle

  Using cached bleach-6.2.0-py3-none-any.whl.metadata (30 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached webencodings-0.5.1-py2.py3-none-any.whl.metadata (2.1 kB)
   ---------------------------------------- 0.0/173.2 kB ? eta -:--:--
   ---------------------------- ----------- 122.9/173.2 kB 3.5 MB/s eta 0:00:01
   ---------------------------------------- 173.2/173.2 kB 3.5 MB/s eta 0:00:00
Using cached bleach-6.2.0-py3-none-any.whl (163 kB)
   ---------------------------------------- 0.0/78.2 kB ? eta -:--:--
   ---------------------------------------- 78.2/78.2 kB 4.5 MB/s eta 0:00:00
Using cached tqdm-4.67.1-py3-none-any.whl (78 kB)
Using cached webencodings-0.5.1-py2.py3-none-any.whl (11 kB)



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# 데이터셋 다운로드
!kaggle competitions download -c dogs-vs-cats

403 Client Error: Forbidden for url: https://www.kaggle.com/api/v1/competitions/data/download-all/dogs-vs-cats
